In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
import json

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

<hr />
<hr />
<hr />

In [2]:
super_srag = spark.read.parquet('gs://ai-covid19-datalake/standard/super-srag/super_srag_v2.parquet')

In [9]:
print('Super SRAG has {} records'.format(super_srag.count()))
super_srag.limit(5).toPandas()

Super SRAG has 3726465 records


,NU_NOTIFIC,CS_SEXO,DT_NASC,AGE_AT_NOTIF,AGE_GROUP,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,SG_UF,CO_MUN_RES,SEM_PRI,SEM_NOT,DIST_PRI_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,CO_MUN_NOT,SURTO_SG,NOSOCOMIAL,AVE_SUINO,VACINA,HOSPITAL,DIST_PRI_INTERNA,SUPORT_VEN,UTI,DIST_PRI_ENTUTI,CLASSI_OUT,CRITERIO,EVOLUCAO,CLASSI_FIN,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,OUTRO_SIN,OUTRO_DES,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,OBES_IMC,OUT_MORBI,MORB_DESC,RAIOX_RES,DIST_PRI_RAIOX,TOMO_RES,DIST_PRI_TOMO,AMOSTRA,TP_AMOSTRA,DT_COLETA,DIST_PRI_COLETA,PP_IF_RESUL,PP_TRA_RESUL,DIST_PRI_TRA,PP_PCR_RESUL,DIST_PRI_PCR,PP_RES_SOR_IGA,PP_RES_SOR_IGM,PP_RES_SOR_IGG,DIST_PRI_SOR,DIST_PRI_IF,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,DIST_PRI_RAIOX_Q,DIST_PRI_TOMO_Q,DIST_PRI_COLETA_Q,DIST_PRI_SOR_Q,DIST_PRI_PCR_Q,DIST_PRI_TRA_Q,DIST_PRI_IF_Q,EPI_WEEK_YEAR,GMR_TRANSIT_STATIONS_AVG,GMR_GROCERY_AND_PHARMACY_AVG,GMR_RETAIL_AND_RECREATION_AVG,GMR_WORKPLACES_PERCENT_AVG,GMR_RESIDENTIAL_PERCENT_AVG,GMR_PARKS_PERCENT_AVG,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_AVG,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_AVG,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_AVG,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_AVG,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_AVG,GMR_PARKS_PERCENT_1WEEK_BEFORE_AVG,GMR_TRANSIT_STATIONS_2WEEKS_AVG,GMR_GROCERY_AND_PHARMACY_2WEEKS_AVG,GMR_RETAIL_AND_RECREATION_2WEEKS_AVG,GMR_WORKPLACES_PERCENT_2WEEKS_AVG,GMR_RESIDENTIAL_PERCENT_2WEEKS_AVG,GMR_PARKS_PERCENT_2WEEKS_AVG,INMET_TEMP_C_AVG,INMET_RELATIVE_AIR_HUMIDITY_AVG,INMET_DAILY_PRECIPT_AVG,INMET_TEMP_C_1WEEK_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_AVG,INMET_DAILY_PRECIPT_1WEEK_BEFORE_AVG,INMET_TEMP_C_2WEEKS_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_AVG,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_AVG,GMR_TRANSIT_STATIONS_Q,GMR_GROCERY_AND_PHARMACY_Q,GMR_RETAIL_AND_RECREATION_Q,GMR_WORKPLACES_PERCENT_Q,GMR_RESIDENTIAL_PERCENT_Q,GMR_PARKS_PERCENT_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_PARKS_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_GROCERY_AND_PHARMACY_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_PARKS_PERCENT_2WEEKS_Q,INMET_TEMP_C_Q,INMET_RELATIVE_AIR_HUMIDITY_Q,INMET_DAILY_PRECIPT_Q,INMET_TEMP_C_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_DAILY_PRECIPT_1WEEK_BEFORE_Q,INMET_TEMP_C_2WEEKS_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_Q,ANO
0,68719768275,M,1967-01-25,54,7,6,4,None,NaN,BA,290240,19,20,7,2021-05-15,BA,291800,None,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,None,None,None,NaN,9,3,2,1,NaN,None,1,1,1,1,None,None,None,NaN,None,None,NaN,2.0,NaN,None,NaN,9,9,NaN,6,None,4,4,4,None,NaN,3,6,6,6,6,6,6,6,6,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021
1,68719899483,M,1959-10-20,61,8,6,4,None,9.0,BA,292050,19,20,8,2021-05-11,BA,292050,None,2.0,2.0,NaN,1,8.0,3.0,2.0,NaN,None,None,None,NaN,3,1,3,1,1.0,ASTENIA E INAPETENCIA,1,1,1,1,None,None,None,NaN,None,6,NaN,1.0,1.0,2021-05-17,6.0,9,9,NaN,7,None,4,4,4,None,NaN,3,4,6,6,6,6,6,6,4,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021
2,77309654484,F,1934-08-17,86,10,5,4,None,NaN,BA,292740,19,19,1,2021-05-09,BA,291920,None,2.0,2.0,NaN,1,0.0,3.0,

<hr />
<hr />
<hr />

In [6]:
super_srag.select(['CLASSI_FIN', 'PP_PCR_RESUL', 'CRITERIO']).printSchema()

root
 |-- CLASSI_FIN: integer (nullable = true)
 |-- PP_PCR_RESUL: string (nullable = true)
 |-- CRITERIO: string (nullable = true)



# Creating filtered data for not covid

In [18]:
filter_not_covid = ((F.col('CLASSI_FIN') ==  1) | (F.col('CLASSI_FIN') ==  2))

In [19]:
super_srag.filter(filter_not_covid).write.parquet('gs://ai-covid19-datalake/trusted/super-srag_filters/super-srag_notcovid_nofilter.parquet', mode='overwrite')
print('Dataset with negative covid cases (without any filter): {}'.format(super_srag.filter(filter_not_covid).count()))

Dataset with negative covid cases (without any filter): 44538


# Creating filtered data for positive covid

In [12]:
filter_positive = (F.col('CLASSI_FIN') == 5)
filter_pcr_positive = ((F.col('PP_PCR_RESUL') ==  '2'))
filter_lab_positive = ((F.col('CRITERIO') ==  '1'))

## covid (without filter)

In [13]:
super_srag.filter(filter_positive).write.parquet('gs://ai-covid19-datalake/trusted/super-srag_filters/super-srag_covid_nofilter.parquet', mode='overwrite')
print('Dataset with positive covid cases (without any filter): {}'.format(super_srag.filter(filter_positive).count()))

Dataset with positive covid cases (without any filter): 2454142


## covid (positive pcr)

In [14]:
super_srag.filter(filter_positive).filter(filter_pcr_positive).write.parquet('gs://ai-covid19-datalake/trusted/super-srag_filters/super-srag_covid_pcrpositive.parquet', mode='overwrite')
print('Dataset with positive covid cases (filtered by positive PCR): {}'.format(super_srag.filter(filter_positive).filter(filter_pcr_positive).count()))

Dataset with positive covid cases (filtered by positive PCR): 1507881


## covid (positive lab)

In [15]:
super_srag.filter(filter_positive).filter(filter_lab_positive).write.parquet('gs://ai-covid19-datalake/trusted/super-srag_filters/super-srag_covid_labpositive.parquet', mode='overwrite')
print('Dataset with positive covid cases (filtered by LAB criteria): {}'.format(super_srag.filter(filter_positive).filter(filter_lab_positive).count()))

Dataset with positive covid cases (filtered by LAB criteria): 2157545


# Creating Super SRAG for reclassification

In [16]:
filter_reclass = (F.col('CLASSI_FIN') ==  4)

In [17]:
super_srag.filter(filter_reclass).write.parquet('gs://ai-covid19-datalake/trusted/super-srag_filters/super-srag_toreclassify.parquet', mode='overwrite')
print('Dataset with positive covid cases (filtered by LAB criteria): {}'.format(super_srag.filter(filter_reclass).count()))

Dataset with positive covid cases (filtered by LAB criteria): 882348


In [22]:
3*3*500

4500